## Packages

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from warnings import filterwarnings
from pprint import pprint
import gc
filterwarnings("ignore")
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

## Global Parameters

In [2]:
ROOT_DIRECTORY = "/home/kaan.aytekin/Thesis"
# Non-feature columns
non_feature_columns = ["simulation_run", "is_accident_simulation", 
                       "accident_location", "accident_start_time", 
                       "accident_duration", "accident_lane", 
                       "prev_detector_detector_number","next_detector_detector_number",
                       "detector_number", "timestamp"
]

## UDFs

In [3]:
def sample_from_array(array,freq):
    array_size = len(array)
    sample_size = int(np.ceil(array_size*freq))
    array_slicer = np.zeros(array_size)
    test_index =  np.random.choice(range(0,array_size),size=sample_size,replace=False)
    array_slicer[test_index] = 1
    return array[array_slicer.astype(bool)]

def kfolds_from_array(array,k,seed=None):
    if seed:
        np.random.seed(seed)
    np.random.shuffle(array)
    array_folds = np.array_split(array,k)
    return array_folds

def simulation_based_k_folds_split(df,k=10,seed=None):
    if seed:
        np.random.seed(seed)
    unique_simulation_combinations = df[["simulation_run","is_accident_simulation","accident_lane"]].drop_duplicates().reset_index(drop=True)
    unique_simulation_combinations = df[["simulation_run","is_accident_simulation","accident_lane"]].drop_duplicates().reset_index(drop=True)
    test_simulation_runs = unique_simulation_combinations.groupby(["is_accident_simulation","accident_lane"]).simulation_run.unique().apply(lambda x: kfolds_from_array(x,k=k))
    test_simulation_runs = test_simulation_runs.reset_index()
    
    for fold_number in range(k):
        complete_test_index = []
        for row in test_simulation_runs.itertuples():
            current_test_index = (
                (df.is_accident_simulation == row.is_accident_simulation)
                &(df.accident_lane == row.accident_lane)
                &(df.simulation_run.isin(row.simulation_run[fold_number]))
            )
            if len(complete_test_index):
                complete_test_index = (complete_test_index | current_test_index)
            else:
                complete_test_index = current_test_index

        train_index = ~complete_test_index
        test_index = complete_test_index
        #df_train = df[~complete_test_index].reset_index(drop=True)
        #df_test = df[complete_test_index].reset_index(drop=True)
        yield train_index, test_index #df_train, df_test

def simulation_based_train_test_split(df, test_size=0.2, seed=None):
    """
    Splits {df} into train and test datasets by their simulation-type with given {test_size}
    """
    if seed:
        np.random.seed(seed)
    unique_simulation_combinations = (
        df[["simulation_run","is_accident_simulation","accident_lane"]].drop_duplicates().reset_index(drop=True)
    )
    test_simulation_runs = (
        unique_simulation_combinations.groupby(
            ["is_accident_simulation", "accident_lane"]
        )
        .simulation_run.unique()
        .apply(lambda x: sample_from_array(x, freq=test_size))
    )
    test_simulation_runs = test_simulation_runs.reset_index()

    complete_test_index = []
    for row in test_simulation_runs.itertuples():
        current_test_index = (
            (df.is_accident_simulation == row.is_accident_simulation)
            & (df.accident_lane == row.accident_lane)
            & (df.simulation_run.isin(row.simulation_run))
        )
        if len(complete_test_index):
            complete_test_index = complete_test_index | current_test_index
        else:
            complete_test_index = current_test_index
    train_index = ~complete_test_index
    test_index = complete_test_index
    #df_train = df[~complete_test_index].reset_index(drop=True)
    #df_test = df[complete_test_index].reset_index(drop=True)
    return train_index, test_index #df_train, df_test

def custom_cross_validation(models_list,performance_metrics_list,df_train,test_size=0.2,repetition_count=5,k_folds=10,seed=None):
    if seed:
        np.random.seed(seed)
    results = []
    for repetition in repetition_count:
        df_train, df_validate = simulation_based_train_test_split(df=df_train,test_size=test_size)
        x_train = df_train[feature_columns]
        y_train = df_train["target"]
        x_validate = df_validate[feature_columns]
        y_validate = df_validate["target"]
        for x,y in simulation_based_k_folds_split:
            for model in models_list:
                model.fit(x_train,y_train)
                y_predicted = model.predict(x_validate)
                for performance_metric in performance_metrics_list:
                    performance_metric(y_validate,y_predicted)

## Data Loading

In [4]:
x_train = pd.read_csv(os.path.join(ROOT_DIRECTORY,"data/thesis_data/x_train.csv"))
y_train = pd.read_csv(os.path.join(ROOT_DIRECTORY,"data/thesis_data/y_train.csv"))

x_test = pd.read_csv(os.path.join(ROOT_DIRECTORY,"data/thesis_data/x_test.csv"))
y_test = pd.read_csv(os.path.join(ROOT_DIRECTORY,"data/thesis_data/y_test.csv"))

In [5]:
x_train.head()

,simulation_run,is_accident_simulation,accident_location,accident_start_time,accident_duration,accident_lane,prev_detector_detector_number,next_detector_detector_number,detector_number,timestamp,...,prev_detector_flow_vehph_lag9,prev_detector_density_vehpkm_lag9,prev_detector_avg_speed_kmph_lag9,prev_detector_section_travel_time_sec_lag9,prev_detector_delay_time_sec_lag9,prev_detector_flow_vehph_lag10,prev_detector_density_vehpkm_lag10,prev_detector_avg_speed_kmph_lag10,prev_detector_section_travel_time_sec_lag10,prev_detector_delay_time_sec_lag10
0,0,0,0,0,0,0,1.0,2.0,1,915,...,1440.0,13.652174,49.2,36.585366,19.343987,1440.0,13.652174,49.2,36.585366,19.343987
1,0,0,0,0,0,0,1.0,2.0,1,930,...,1440.0,13.652174,49.2,36.585366,19.343987,1440.0,13.652174,49.2,36.585366,19.343987
2,0,0,0,0,0,0,1.0,2.0,1,945,...,1440.0,13.652174,49.2,36.585366,19.343987,1440.0,13.652174,49.2,36.585366,19.343987
3,0,0,0,0,0,0,1.0,2.0,1,960,...,1440.0,13.652174,49.2,36.585366,19.343987,1440.0,13.652174,49.2,36.585366,19.343987
4,0,0,0,0,0,0,1.0,2.0,1,975,...,1440.0,13.652174,49.2,36.585366,19.343987,1440.0,13.652174,49.2,36.585366,19.343987


## Feature Selection & Preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

### Top Features

In [ ]:
top_features_df = pd.read_csv(os.path.join(ROOT_DIRECTORY,"data/thesis_data/top_features.csv"))
selected_features = top_features_df.feature.to_list()

In [ ]:
non_feature_columns = [
    "simulation_run", 
    "is_accident_simulation", 
    #"accident_location", 
    #"accident_start_time", 
    #"accident_duration", 
    "accident_lane", 
    #"prev_detector_detector_number",
    #"next_detector_detector_number",
    #"detector_number", 
    #"timestamp"
]

### Configs + Features

In [ ]:
x_train = x_train[non_feature_columns + selected_features]
x_test = x_test[selected_features]

### One Hot Encoding for "Accident Lane"

In [ ]:
x_train_accident_lane_categorical = x_train[["accident_lane"]]

one_hot_encoder = OneHotEncoder(drop="first")
one_hot_encoder.fit(x_train_accident_lane_categorical)

x_train_accident_lane_df = pd.DataFrame(one_hot_encoder.transform(x_train_accident_lane_categorical).toarray())
x_train_accident_lane_df.columns = one_hot_encoder.get_feature_names(["accident_lane"])

In [ ]:
x_train = pd.concat([x_train_accident_lane_df,x_train],axis = 1)

In [ ]:
df_train = x_train.copy()
df_train["target"] = y_train

In [ ]:
df_train.columns

In [ ]:
FEATURE_COLUMNS =  ["accident_lane_1", "accident_lane_2", "accident_lane_3", 'accident_lane'] + selected_features

## ML Packages

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge, ARDRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as xgb
from sklearn.metrics import make_scorer,get_scorer, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from tune_sklearn import TuneSearchCV
from scipy.stats import uniform, randint
# Reference: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
xg_train = xgb.DMatrix(x_train.values, y_train.values)
xg_test = xgb.DMatrix(x_test.values, y_test.values)

## Cross Validation: Hyper Parameter Tuning & Model Selection

In [ ]:
def custom_cross_validation(
    pipe,
    param_grid,
    performance_metric,
    df,
    test_size=0.2,
    repetition_count=5,
    k_folds=10,
    seed=None,
    n_jobs=-1,
    n_iter=10
    ):
    from sklearn.metrics import get_scorer

    if seed:
        np.random.seed(seed)
    results = []
    for repetition in range(repetition_count):
        repetition_results = {"repetition": repetition}
        train_index, validation_index = simulation_based_train_test_split(
            df=df, test_size=test_size
        )
        df_train, df_validation = df[train_index], df[validation_index]
        x_train = df_train[FEATURE_COLUMNS]
        y_train = df_train["target"]
        x_validate = df_validation[FEATURE_COLUMNS]
        y_validate = df_validation["target"]
        inner_cv = list(simulation_based_k_folds_split(df=df_train, k=k_folds))
        #bayesian_optimizer = TuneSearchCV(
        optimizer = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=param_grid,
            scoring=performance_metric,
            n_jobs = n_jobs,
            cv=inner_cv,
            n_iter = n_iter,
            verbose=2,
            return_train_score=True,
            #search_optimization="random",#"bayesian",
        )
        cv_results = optimizer.fit(x_train, y_train)
        #cv_results = optimizer.cv_results_
        repetition_results["cv_results"] = cv_results
        best_model = optimizer.best_estimator_
        validation_predictions = best_model.predict(x_validate)
        scorer_function = get_scorer(performance_metric)._score_func
        score = scorer_function(y_validate, validation_predictions)
        repetition_results["best_model_validation_score"] = score
        results.append(repetition_results)
    return results

In [ ]:
tuning_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("classifier", RandomForestRegressor())
]
)

param_grid = [
    {
        "classifier": [RandomForestRegressor()], 
        "classifier__n_estimators": [100,500,1000,1500],#randint(100,1001),#(100,1000),
        "classifier__min_samples_leaf": [1,10,50,100,200,500],#randint(1,201),#(1,200),
        "classifier__max_features": ["auto", "sqrt", "log2", 1/3],
        "classifier__n_jobs": [-1],
    },
    #{
    #    "classifier": [GradientBoostingRegressor()], #lightGBM or XGBoost for future?
    #    #"classifier": [HistGradientBoostingRegressor()],
    #    "classifier__loss": ["ls", "lad", "huber", "quantile"],
    #    #"classifier__loss": ["least_squares", "least_absolute_deviation", "poisson"],
    #    "classifier__learning_rate": uniform(0,1),# (0,1),
    #    "classifier__n_estimators": [100,500,1000,1500],#(10,500),
    #    "classifier__subsample": uniform(0,1),#(0,1),
    #    "classifier__criterion": ["friedman_mse","mse","mae"],
    #    "classifier__min_samples_leaf": [1,10,50,100,200,500],#(1,200),
    #    "classifier__max_features": ["auto", "sqrt", "log2", 1/3],
    #},
    {
        "classifier": [xgb.XGBRegressor(objective='reg:squarederror')],
        "classifier__n_estimators": [100,500,1000,1500],
        "classifier__learning_rate": [0.01,0.03,0.05,0.1,0.2,0.3],
        "classifier__n_jobs": [-1],
        "classifier__min_child_weight": [1,10,50,100,200,500],
        "classifier__reg_alpha": [0,0.05,0.1]
    },
    {
        "classifier": [LinearRegression()],
        "classifier__n_jobs": [-1],
    },
    {
        "classifier": [Ridge()],
        "classifier__alpha": uniform(0,10),#(0,100),
        
    },
    {
        "classifier": [Lasso()],
        "classifier__alpha": uniform(0,10),#(0,100),
    },
    {
        "classifier": [BayesianRidge()],
        "classifier__alpha_1": uniform(0,1),#(0,1),
        "classifier__alpha_2": uniform(0,1),#(0,1),
    },
]

In [ ]:
param_grid = [
    {
        "classifier": [xgb.XGBRegressor(objective='reg:squarederror')],
        "classifier__n_estimators": [100,500,1000,1500],
        "classifier__learning_rate": [0.01,0.03,0.05,0.1,0.2,0.3],
        "classifier__n_jobs": [-1],
        #"classifier__min_child_weight": [1,10,50,100,200,500],
        #"classifier__reg_alpha": [0,0.05,0.1]
    },
]

In [ ]:
df_train_sample = df_train.sample(n=1000)

In [ ]:
df_train_sample.head()

In [ ]:
%%time
results = custom_cross_validation(
    pipe=tuning_pipeline,
    param_grid=param_grid,
    performance_metric = "neg_mean_squared_error",
    df=df_train_sample,#df_train,
    test_size=0.2,
    repetition_count=5,
    k_folds=5,
    seed=5,
    n_jobs=50,
    n_iter=50
)

In [ ]:
cv = results[0]["cv_results"]

In [ ]:
cv.best_params_

## Best Model

In [ ]:
best_result = np.Inf
best_model = None
for result in results:
    if result["best_model_validation_score"] < best_result:
        best_model = results[0]["cv_results"].best_params_

## Model Inspection

## Test Performance

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x_train)
x_test_scaled = min_max_scaler.transform(x_test)
y_pred = best_model.predict(x_test_scaled)

test_mse = mean_squared_error(y_test,y_pred)

## Prediction plotting

In [ ]:
results[0].keys()
#results: list
#results[0]: dict('repetition', 'cv_results', 'best_model_test_scores')
#results[0]["repetition"]: int
#results[0]["cv_results"]: sklearn.model_selection._search.RandomizedSearchCV
#results[0]["cv_results"].best_params_
#results[0]["best_model_validation_score"]: float